In [6]:
import pandas as pd
import numpy as np
import subprocess
import os

In [7]:
# 配置运行路径，需要师哥你自己修改一下
workdir_omic = '/Users/dongjiacheng/Desktop/coder/mtd/code/analysis_module/'

#### 调用R脚本，对输入的基因表达矩阵，进行差异分析，生成差异分析文件

In [8]:
def run_deseq(input_path, output_path, repetition):
    """
    运行R脚本，进行差异分析

    Args:
        input_path: 输入文件路径
        output_path: 输出文件路径
        repetition: 样本重复次数
    
    """

    # R脚本的路径，需要师哥你改路径
    script_path = os.path.join(workdir_omic, 'differential_analysis/Deseq2.R')

    cmd = [
        'Rscript', script_path,
        '--input', input_path,
        '--output', output_path,
        '--repetition', str(repetition),
    ]
    
    # 执行R脚本并捕获输出
    try:
        result = subprocess.run(cmd, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        return result.stdout
    except subprocess.CalledProcessError as e:
        return e.stderrs
    
# 示例调用
# output_deseq = run_deseq("input_file/expression_matrix_deseq2.csv", "output_file/deseq2.tsv", 3)

#### 基于Python的Plotly包，对差异分析结果进行可视化

In [33]:
df_deseq2 = pd.read_csv("output_file/deseq2.tsv", sep="\t")
df_deseq2.head()

,Gene,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
0,MYCTH_2114025,84.477576,1.769935,0.295154,5.996652,2.014266e-09,4.217154e-09
1,MYCTH_2293935,27.971419,-0.833626,0.474323,-1.757506,7.883160e-02,9.537086e-02
2,MYCTH_2293936,13.674138,-2.335144,0.702318,-3.324910,8.844695e-04,1.288265e-03
3,MYCTH_2051335,331.950405,-0.889314,0.131604,-6.757494,1.403993e-11,3.266067e-11
4,MYCTH_2293939,805.966778,1.843547,0.107909,17.084273,1.943496e-65,1.805006e-64


In [ ]:
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go

def plot_volcano(df_deseq2, width=1200, height=900 , p_threshold=0.05, logFC_threshold=1, color_schemes = 1, bubble_size=5, opacity=0.8, up_donw_info='yes', x_ca='yes',y_ca='yes'):
    """根据差异分析结果绘制火山图
    """
    # 预处理
    df_deseq2 = df_deseq2[['Gene', 'log2FoldChange', 'padj']].copy()
    df_deseq2.columns = ['Symbol', 'logFC', 'P.adjust']
    df_deseq2.dropna(inplace=True)

    # 对logFC范围进行修正
    if x_ca == 'yes':
        df_deseq2['logFC'] = np.clip(df_deseq2['logFC'], -6, 6)
        
    df_deseq2['P.adjust'] = df_deseq2['P.adjust'].replace(0, 1e-300)
    pvalue = -np.log10(df_deseq2['P.adjust'])

    # 对y轴范围进行修正
    if y_ca == 'yes':
        pvalue= np.clip(pvalue, 0, 100)
    
    # 提取数据
    gene_names = df_deseq2['Symbol'].values
    logFC = df_deseq2['logFC'].values

    # 根据阈值筛选差异显著的基因，给差异最显著的基因添加标签
    significant = (np.abs(logFC) > logFC_threshold) & (df_deseq2['P.adjust'] < p_threshold)
    upregulated = significant & (logFC > 0)
    downregulated = significant & (logFC < 0)
    nonsignificant = ~significant

    # 颜色设置
    colors = {
        1: ('#f08d1a', '#7fa4ca'),
        2: ('#f26c6a', '#54a857'),
        3: ('#c42121', '#15609b'),
        4: ('#df7415', '#3d9241')
    }
    up_color, down_color = colors.get(color_schemes, ('#f08d1a', '#7fa4ca'))

    # 绘图
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=logFC[upregulated], y=pvalue[upregulated], mode='markers',
                            marker=dict(color=up_color, size=bubble_size, sizemode='area',symbol='circle',opacity=opacity,
                                        # line_dict=1
                                        line=dict(color='black',width=0.4)), name='Up',
                            text=gene_names[upregulated]))
    fig.add_trace(go.Scatter(x=logFC[downregulated], y=pvalue[downregulated], mode='markers',
                            marker=dict(color=down_color, size=bubble_size, sizemode='area',symbol='circle',opacity=opacity,
                                        line=dict(color='black',width=0.4)), name='Down',
                            text=gene_names[downregulated]))
    fig.add_trace(go.Scatter(x=logFC[nonsignificant], y=pvalue[nonsignificant], mode='markers',
                            marker=dict(color='#A9A9A9', size=bubble_size, sizemode='area',symbol='circle',opacity=opacity,
                                        line=dict(color='black',width=0.4)), name='Nonsignificant',
                            text=gene_names[nonsignificant]))

    if up_donw_info == 'yes':
    # 计算上调和下调基因的数量
        upregulated_num = np.sum(upregulated)
        downregulated_num = np.sum(downregulated)
    
        # 在图像中右上角添加上调基因的数量注释
        fig.add_annotation(xref="paper", yref="paper",
                        x=1, y=1, showarrow=False,
                        xanchor='right', yanchor='top',
                        text="Up: {} genes".format(upregulated_num),
                        font=dict(size=16),
                        align="right",
                        bgcolor="white",
                        borderpad=4)
        
        # 在图像中左上角添加下调基因的数量注释
        fig.add_annotation(xref="paper", yref="paper",
                        x=0, y=1, showarrow=False,
                        xanchor='left', yanchor='top',
                        text="Down: {} genes".format(downregulated_num),
                        font=dict(size=16),
                        align="left",
                        bgcolor="white",
                        borderpad=4)
        
    # 添加火山图的阈值线
    x_min = np.min(logFC)
    x_max = np.max(logFC)
    fig.update_layout(shapes=[
        dict(type="line", x0=x_min, x1=x_max, y0=-np.log10(p_threshold), y1=-np.log10(p_threshold), line=dict(color="Black", width=1, dash="dash")),
        dict(type="line", x0=logFC_threshold, x1=logFC_threshold, y0=0, y1=max(pvalue)+10, line=dict(color="Black", width=1, dash="dash")),
        dict(type="line", x0=-logFC_threshold, x1=-logFC_threshold, y0=0, y1=max(pvalue)+10, line=dict(color="Black", width=1, dash="dash"))
    ])

    # 设置图像布局，并限制 y 轴的范围
    fig.update_layout(
        xaxis_title='log2 Fold Change',
        yaxis_title='-log10(p-value)',
        title='DE Analysis Volcano Plot',
        template="plotly_white",
        height=height,
        width=width,
    )

    if y_ca == 'yes':
        fig.update_layout(yaxis_range=[0, 110])

    # 保存上调和下调基因列表
    df_deseq2[upregulated]['Symbol'].to_csv(workdir_omic + 'differential_analysis/output_file/upregulated_genes.txt', index=False, header=False)
    df_deseq2[downregulated]['Symbol'].to_csv(workdir_omic + 'differential_analysis/output_file/downregulated_genes.txt', index=False, header=False)

    # 保存为png，scale设置为4
    fig.write_image(workdir_omic + "differential_analysis/output_file/volcano.png", scale=4)

    # 测试用
    return fig

    # 方案1:将fig对象转为json
    # fig_json = pio.to_json(fig)
    # return fig_json

    # 方案2:将fig转为html格式，返回html代码
    # fig_html = plot(fig, output_type='div', include_plotlyjs=False)        
    # return fig_html

    # 方案3:将fig转为html格式，保存为html文件
    # fig.write_html("./output-file/kegg.html")
    # return "kegg.html"

# 示例调用
plot_volcano(df_deseq2, 
             p_threshold=0.05, 
             logFC_threshold=1, 
             opacity = 0.8,
             width=1000, height=800,
             color_schemes = 1,
             bubble_size=10,
             up_donw_info='yes',
             x_ca='no',
             y_ca='no'
             )